In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from datasets import Dataset

In [6]:
df = pd.read_csv('Conversation.csv')

In [7]:
# Combine question-answer pairs into a dialogue format for training
df['input_text'] = df['question'] + " " + df['answer']
dataset = Dataset.from_pandas(df[['input_text']])

In [10]:
# Initialize the tokenizer and model
model_name = "gpt2"  # Choose a small model like distilbert, bert, or gpt2
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [12]:
# Set padding token
tokenizer.pad_token = tokenizer.eos_token

In [13]:
# Resize model embeddings to account for the added padding token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [14]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 3725/3725 [00:00<00:00, 6064.91 examples/s]


In [15]:
# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank decomposition
    lora_alpha=32,  # Alpha scaling factor
    target_modules=["c_attn", "c_proj"],  # Apply LoRA to attention layers in GPT-2
    lora_dropout=0.1,  # Dropout rate
)

In [17]:
# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

c:\Users\ENVY 13TH GEN\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora-finetuned-model",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
)

In [19]:
# Initialize Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [20]:
# Start training
trainer.train()

  1%|          | 10/1398 [00:22<46:22,  2.00s/it] 

{'loss': 4.1873, 'grad_norm': 0.9092060327529907, 'learning_rate': 4.964234620886982e-05, 'epoch': 0.02}


  1%|▏         | 20/1398 [00:44<49:48,  2.17s/it]

{'loss': 4.1091, 'grad_norm': 1.2194467782974243, 'learning_rate': 4.928469241773963e-05, 'epoch': 0.04}


  2%|▏         | 30/1398 [01:00<34:52,  1.53s/it]

...

 99%|█████████▉| 1390/1398 [48:44<00:18,  2.27s/it]

{'loss': 3.1544, 'grad_norm': 2.168179512023926, 'learning_rate': 2.861230329041488e-07, 'epoch': 2.98}


100%|██████████| 1398/1398 [49:02<00:00,  2.10s/it]

{'train_runtime': 2942.4181, 'train_samples_per_second': 3.798, 'train_steps_per_second': 0.475, 'train_loss': 3.2285721237226275, 'epoch': 3.0}


TrainOutput(global_step=1398, training_loss=3.2285721237226275, metrics={'train_runtime': 2942.4181, 'train_samples_per_second': 3.798, 'train_steps_per_second': 0.475, 'total_flos': 371952702259200.0, 'train_loss': 3.2285721237226275, 'epoch': 3.0})

In [21]:
# Save the fine-tuned model and tokenizer
lora_model.save_pretrained("./lora-finetuned-model")
tokenizer.save_pretrained("./lora-finetuned-model")


('./lora-finetuned-model\\tokenizer_config.json',
 './lora-finetuned-model\\special_tokens_map.json',
 './lora-finetuned-model\\vocab.json',
 './lora-finetuned-model\\merges.txt',
 './lora-finetuned-model\\added_tokens.json',
 './lora-finetuned-model\\tokenizer.json')